# EEG Motor Imagery Classification Example

This notebook demonstrates the complete pipeline for analyzing EEG motor imagery data from the PhysioNet EEGMMIDB dataset.

## Overview
1. Data Loading and Preprocessing
2. Feature Extraction (CSP)
3. Classification (LDA)
4. Visualization and Evaluation

In [ ]:
# Import necessary libraries
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
from preprocessing import (
    preprocess_pipeline,
    get_motor_imagery_event_dict,
    get_recommended_runs
)
from features import extract_csp_features, compute_band_power
from classification import (
    create_csp_lda_pipeline,
    train_and_evaluate,
    compare_classifiers
)
from visualization import (
    plot_raw_psd,
    plot_csp_patterns,
    plot_confusion_matrix,
    plot_roc_curve
)

%matplotlib inline

## 1. Configuration

In [ ]:
# Configuration
SUBJECT = 1  # Subject number (1-109)
TASK_TYPE = 'imagery_left_right'  # Type of motor imagery task
APPLY_ICA = False  # Whether to apply ICA for artifact removal

# Get recommended runs and event dictionary
runs = get_recommended_runs(TASK_TYPE)
event_id = get_motor_imagery_event_dict('left_right_fist')

print(f"Subject: {SUBJECT}")
print(f"Task Type: {TASK_TYPE}")
print(f"Runs: {runs}")
print(f"Events: {event_id}")

## 2. Data Loading and Preprocessing

In [ ]:
# Load and preprocess data
epochs, raw = preprocess_pipeline(
    subject=SUBJECT,
    runs=runs,
    event_id=event_id,
    apply_ica=APPLY_ICA,
    l_freq=7.0,
    h_freq=30.0,
    notch_freq=60.0,
    verbose=True
)

print(f"\nEpochs shape: {epochs.get_data().shape}")
print(f"Number of epochs: {len(epochs)}")
print(f"Number of channels: {len(epochs.ch_names)}")
print(f"Sampling rate: {epochs.info['sfreq']} Hz")

## 3. Visualize Power Spectral Density

In [ ]:
# Plot PSD
plot_raw_psd(raw, fmin=0.5, fmax=40.0, show=True)

## 4. Feature Extraction with CSP

In [ ]:
# Extract CSP features
csp, features = extract_csp_features(epochs, n_components=4)

print(f"CSP Features shape: {features.shape}")
print(f"Number of CSP components: {csp.n_components}")

## 5. Visualize CSP Patterns

In [ ]:
# Plot CSP patterns
plot_csp_patterns(csp, epochs.info, n_components=4, show=True)

## 6. Classification with Cross-Validation

In [ ]:
# Create CSP+LDA pipeline
pipeline = create_csp_lda_pipeline(n_components=4)

# Perform cross-validation
scores = train_and_evaluate(
    epochs,
    pipeline,
    cv=5,
    scoring='accuracy',
    verbose=True
)

## 7. Train/Test Split Evaluation

In [ ]:
# Split data
n_epochs = len(epochs)
train_size = int(0.8 * n_epochs)

indices = np.arange(n_epochs)
np.random.seed(42)
np.random.shuffle(indices)

train_indices = indices[:train_size]
test_indices = indices[train_size:]

train_epochs = epochs[train_indices]
test_epochs = epochs[test_indices]

print(f"Training set: {len(train_epochs)} epochs")
print(f"Test set: {len(test_epochs)} epochs")

In [ ]:
from classification import evaluate_on_test_set

# Train and evaluate
results = evaluate_on_test_set(
    train_epochs,
    test_epochs,
    pipeline,
    verbose=True
)

## 8. Visualize Results

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(
    results['y_true'],
    results['y_pred'],
    labels=['Left Fist', 'Right Fist'],
    normalize=False,
    show=True
)

In [ ]:
# Plot ROC curve
plot_roc_curve(
    results['y_true'],
    results['y_proba'][:, 1],
    show=True
)

## 9. Compare Different Classifiers

In [ ]:
# Compare classifiers
comparison_results = compare_classifiers(epochs, cv=5, verbose=True)

In [ ]:
from visualization import plot_classifier_comparison

# Visualize comparison
plot_classifier_comparison(comparison_results, show=True)

## 10. Analyze Frequency Band Power

In [ ]:
# Compute band power
freq_bands = {
    'mu': (8, 13),
    'beta': (13, 30)
}

band_power, band_names = compute_band_power(
    epochs,
    freq_bands=freq_bands,
    normalize=True
)

print(f"Band power shape: {band_power.shape}")
print(f"Bands: {band_names}")

In [ ]:
# Plot band power for motor cortex channels
motor_channels = ['C3', 'Cz', 'C4']
channel_indices = [epochs.ch_names.index(ch) for ch in motor_channels]

fig, axes = plt.subplots(1, len(band_names), figsize=(12, 4))

for i, band_name in enumerate(band_names):
    for j, ch_idx in enumerate(channel_indices):
        power = band_power[:, ch_idx, i]
        axes[i].hist(power, alpha=0.5, label=motor_channels[j], bins=20)
    
    axes[i].set_xlabel('Normalized Power')
    axes[i].set_ylabel('Count')
    axes[i].set_title(f'{band_name.capitalize()} Band')
    axes[i].legend()
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
1. Loading and preprocessing EEG data from the PhysioNet EEGMMIDB dataset
2. Extracting CSP features for motor imagery classification
3. Training and evaluating classifiers with cross-validation
4. Visualizing results and comparing different approaches

The pipeline can be easily adapted for:
- Different subjects
- Different motor imagery tasks
- Custom preprocessing parameters
- Alternative feature extraction methods
- Different classification algorithms